## Part 1

In [127]:
import numpy as np
import pandas as pd
import re
from datetime import datetime

In [128]:
data = [str(x) for x in open("input.txt").read().splitlines()]

In [129]:
dates = [re.findall(r'\[(.*)\]', entry)[0] for entry in data]

In [130]:
rest = [re.findall(r'\] (.*)$', entry)[0] for entry in data]

In [131]:
guard_ids = []
for i in range(0,len(rest)):
    entry = rest[i]
    if re.findall(r'(\d+)', entry) == []:
        guard_ids.append(np.nan)
    else :
        guard_ids.append(int(re.findall(r'(\d+)', entry)[0]))

In [132]:
guard_shift_df = pd.DataFrame(
    {'date': dates,
     'guard_id': guard_ids,
     'note': rest
    })

In [133]:
shift_sorted = guard_shift_df.sort_values(by = 'date').reset_index(drop = True)
shift_sorted['guard_id'].fillna(method = 'ffill', inplace = True)

In [134]:
shift_sorted['minute'] = shift_sorted.date.str[-2:]
shift_sorted['minute'] = pd.to_numeric(shift_sorted['minute'])

In [135]:
shift_sorted['time_asleep'] = 0
shift_sorted.loc[shift_sorted['note'] == 'wakes up', 'time_asleep'] = shift_sorted.minute.diff()
shift_sorted.head(10)


,date,guard_id,note,minute,time_asleep
0,1518-02-24 23:58,853.0,Guard #853 begins shift,58,0.0
1,1518-02-25 00:20,853.0,falls asleep,20,0.0
2,1518-02-25 00:43,853.0,wakes up,43,23.0
3,1518-02-25 23:56,3499.0,Guard #3499 begins shift,56,0.0
4,1518-02-26 00:11,3499.0,falls asleep,11,0.0
5,1518-02-26 00:19,3499.0,wakes up,19,8.0
6,1518-02-26 00:37,3499.0,falls asleep,37,0.0
7,1518-02-26 00:44,3499.0,wakes up,44,7.0
8,1518-02-26 00:53,3499.0,falls asleep,53,0.0
9,1518-02-26 00:58,3499.0,wakes up,58,5.0


In [136]:
shift_sorted[['guard_id', 'time_asleep']].groupby(by = 'guard_id').sum().sort_values(by = 'time_asleep', ascending = False).head(1)

,time_asleep
guard_id,
727.0,444.0


## Guard 727 sleeps the most
444 minutes

In [137]:
guard_727 = shift_sorted.loc[(shift_sorted['guard_id'] == 727.0) & ((shift_sorted['note'] == 'wakes up') | (shift_sorted['note'] == 'falls asleep')), :]

In [138]:
a = np.zeros(60)

In [139]:
guard_interest_list = guard_727['minute'].tolist()
for i in range(1,len(guard_interest_list)-1, 2):
    start = guard_interest_list[i-1]
    stop = guard_interest_list[i]
    a[start:stop] += 1


In [140]:
np.argmax(a)

29

### What is the ID of the guard you chose multiplied by the minute you chose?

In [141]:
29 * 727

21083

## Part one answer is 21,083